# Data Set

In [2]:
# https://benchmark.ini.rub.de/gtsrb_dataset.html#Downloads
# https://www.kaggle.com/meowmeowmeowmeowmeow/gtsrb-german-traffic-sign

# Custom Yolo - 표지판 인식 실습

In [53]:
import pandas as pd
import cv2
import os

## 데이터셋
---

### GTSRB - German Traffic Sign Recognition Benchmark

![](https://production-media.paperswithcode.com/datasets/GTSRB-0000000633-9ce3c5f6_Dki5Rsf.jpg)

* 필요데이터
  * Train.csv
  * Train.zip

In [4]:
from google.colab import drive
drive.mount('/gdrive')

Mounted at /gdrive


In [5]:
! ls /gdrive/MyDrive/딥러닝/Lecture_deeplearning_basic-main/실습/dataset/yolo-traffic-sign

darknet  traffic-train-yolo.cfg  Train.csv  Train.zip


## YOLO Format 제작
---

원본 데이터의 csv 파일을 욜로 포멧 파일로 변환합니다.

욜로 포멧에는 아래와 같은 값들이 필요합니다.

* ClassID
* Center X
* Center Y
* Width
* Height

In [6]:
annot = pd.read_csv("/gdrive/MyDrive/딥러닝/Lecture_deeplearning_basic-main/실습/dataset/yolo-traffic-sign/Train.csv")
annot.head()

,Width,Height,Roi.X1,Roi.Y1,Roi.X2,Roi.Y2,ClassId,Path
0,27,26,5,5,22,20,20,Train/20/00020_00000_00000.png
1,28,27,5,6,23,22,20,Train/20/00020_00000_00001.png
2,29,26,6,5,24,21,20,Train/20/00020_00000_00002.png
3,28,27,5,6,23,22,20,Train/20/00020_00000_00003.png
4,28,26,5,5,23,21,20,Train/20/00020_00000_00004.png


* 클래스 값 초기화

In [7]:
annot['Categoryid'] = 0
annot.head()

,Width,Height,Roi.X1,Roi.Y1,Roi.X2,Roi.Y2,ClassId,Path,Categoryid
0,27,26,5,5,22,20,20,Train/20/00020_00000_00000.png,0
1,28,27,5,6,23,22,20,Train/20/00020_00000_00001.png,0
2,29,26,6,5,24,21,20,Train/20/00020_00000_00002.png,0
3,28,27,5,6,23,22,20,Train/20/00020_00000_00003.png,0
4,28,26,5,5,23,21,20,Train/20/00020_00000_00004.png,0


* 클래스 묶기

```
Prohibitory = [0, 1, 2, 3, 4, 5, 7, 8, 9, 10, 15, 16]
Danger = [11, 18, 19, 20, 21, 22, 23, 24, 25, 26, 27, 28, 29, 30, 31]
Mandatory = [33, 34, 35, 36, 37, 38, 39, 40]
Other = [6, 12, 13, 14, 17, 32, 41, 42]

```



In [8]:
class_names = ["Prohibitory", "Danger", "Mandatory", "Other"]

In [9]:
Prohibitory = [0, 1, 2, 3, 4, 5, 7, 8, 9, 10, 15, 16]
Danger = [11, 18, 19, 20, 21, 22, 23, 24, 25, 26, 27, 28, 29, 30, 31]
Mandatory = [33, 34, 35, 36, 37, 38, 39, 40]
Other = [6, 12, 13, 14, 17, 32, 41, 42]

In [10]:
# Classid가 Prohibitory 값에 속하면 Categoryid를 0으로
# Classid가 Danger 값에 속하면 Categoryid를 1으로
# Classid가 Mandatory 값에 속하면 Categoryid를 2으로
# Classid가 Other 값에 속하면 Categoryid를 3으로

annot.loc[annot['ClassId'].isin(Prohibitory), 'Categoryid'] = 0
annot.loc[annot['ClassId'].isin(Danger), 'Categoryid'] = 1
annot.loc[annot['ClassId'].isin(Mandatory), 'Categoryid'] = 2
annot.loc[annot['ClassId'].isin(Other), 'Categoryid'] = 3

annot.head()

,Width,Height,Roi.X1,Roi.Y1,Roi.X2,Roi.Y2,ClassId,Path,Categoryid
0,27,26,5,5,22,20,20,Train/20/00020_00000_00000.png,1
1,28,27,5,6,23,22,20,Train/20/00020_00000_00001.png,1
2,29,26,6,5,24,21,20,Train/20/00020_00000_00002.png,1
3,28,27,5,6,23,22,20,Train/20/00020_00000_00003.png,1
4,28,26,5,5,23,21,20,Train/20/00020_00000_00004.png,1


* **상대적** 위치 구하기
  * centerX : 네모박스 중앙 X 좌표값
  * centerY : 네모박스 중앙 Y 좌표값
  * width : 네모박스의 가로 사이즈
  * height : 네모박스의 세로 사이즈

In [11]:
# 욜로형식 좌표값 변환
annot['centerX'] = (annot['Roi.X1'] + annot['Roi.X2']) / 2
annot['centerY'] = (annot['Roi.Y1'] + annot['Roi.Y2']) / 2
annot['width'] = (annot['Roi.X2'] - annot['Roi.X1'])
annot['height'] = (annot['Roi.Y2'] - annot['Roi.Y1'])

# 이미지에 대한 상대적 비율로 변환
annot['centerX'] = annot['centerX'] / annot['Width']
annot['centerY'] = annot['centerY'] / annot['Height']
annot['width'] = annot['width'] / annot['Width']
annot['height'] = annot['height'] / annot['Height']

annot.head()

,Width,Height,Roi.X1,Roi.Y1,Roi.X2,Roi.Y2,ClassId,Path,Categoryid,centerX,centerY,width,height
0,27,26,5,5,22,20,20,Train/20/00020_00000_00000.png,1,0.500000,0.480769,0.629630,0.576923
1,28,27,5,6,23,22,20,Train/20/00020_00000_00001.png,1,0.500000,0.518519,0.642857,0.592593
2,29,26,6,5,24,21,20,Train/20/00020_00000_00002.png,1,0.517241,0.500000,0.620690,0.615385
3,28,27,5,6,23,22,20,Train/20/00020_00000_00003.png,1,0.500000,0.518519,0.642857,0.592593
4,28,26,5,5,23,21,20,Train/20/00020_00000_00004.png,1,0.500000,0.500000,0.642857,0.615385


* 필요없는 열 제거

## 만들어야 할것
----
1. 이미지-정답 세트
2. `.txt` (train, test 각각)
3. `.names`
4. `.data`
5. `.cfg`

### 1. 이미지-정답 세트

* 이미지 데이터셋 확보

In [12]:
! cp /gdrive/MyDrive/딥러닝/Lecture_deeplearning_basic-main/실습/dataset/yolo-traffic-sign/Train.zip .

In [13]:
! unzip Train.zip

스트리밍 출력 내용이 길어서 마지막 5000줄이 삭제되었습니다.
 extracting: Train/5/00005_00053_00010.png  
 extracting: Train/5/00005_00053_00011.png  
 extracting: Train/5/00005_00053_00012.png  
 extracting: Train/5/00005_00053_00013.png  
 extracting: Train/5/00005_00053_00014.png  
 extracting: Train/5/00005_00053_00015.png  
 extracting: Train/5/00005_00053_00016.png  
 extracting: Train/5/00005_00053_00017.png  
 extracting: Train/5/00005_00053_00018.png  
 extracting: Train/5/00005_00053_00019.png  
 extracting: Train/5/00005_00053_00020.png  
 extracting: Train/5/00005_00053_00021.png  
 extracting: Train/5/00005_00053_00022.png  
 extracting: Train/5/00005_00053_00023.png  
 extracting: Train/5/00005_00053_00024.png  
 extracting: Train/5/00005_00053_00025.png  
 extracting: Train/5/00005_00053_00026.png  
 extracting: Train/5/00005_00053_00027.png  
 extracting: Train/5/00005_00053_00028.png  
 extracting: Train/5/00005_00053_00029.png  
 extracting: Train/5/00005_00054_00000.png  
 extracting: Train/

* 정답 데이터셋 생성

In [14]:
paths = []
for idx, data in annot.iterrows():
  
  txt_file = data['Path'].split('.')[0] +'.txt'
  categoryid = str(data['Categoryid']) + ' ' 
  centerx = str(data['centerX']) + ' ' 
  centery = str(data['centerY']) +' '
  width = str(data['width']) + ' ' 
  height = str(data['height'])
  line = categoryid+centerx+centery+width+height 
  with open(txt_file, 'w') as txt:
    txt.write(line)
  img_path = data['Path'] + '\n'
  paths.append(img_path)
  print(idx, txt_file, line)

스트리밍 출력 내용이 길어서 마지막 5000줄이 삭제되었습니다.
34209 Train/35/00035_00000_00010.txt 2 0.5 0.5108695652173914 0.7391304347826086 0.7608695652173914
34210 Train/35/00035_00000_00011.txt 2 0.5 0.5104166666666666 0.782608695652174 0.7708333333333334
34211 Train/35/00035_00000_00012.txt 2 0.5 0.5 0.782608695652174 0.7551020408163265
34212 Train/35/00035_00000_00013.txt 2 0.5 0.5102040816326531 0.7872340425531915 0.7755102040816326
34213 Train/35/00035_00000_00014.txt 2 0.5 0.5 0.7916666666666666 0.7959183673469388
34214 Train/35/00035_00000_00015.txt 2 0.5 0.5 0.8076923076923077 0.8076923076923077
34215 Train/35/00035_00000_00016.txt 2 0.509090909090909 0.5 0.8 0.8148148148148148
34216 Train/35/00035_00000_00017.txt 2 0.49107142857142855 0.5089285714285714 0.8035714285714286 0.8035714285714286
34217 Train/35/00035_00000_00018.txt 2 0.5 0.5084745762711864 0.8275862068965517 0.8135593220338984
34218 Train/35/00035_00000_00019.txt 2 0.5083333333333333 0.5 0.8166666666666667 0.8333333333333334
34219 Train

### 2. `.txt` (train, test 각각) 

* train-test 분리

In [15]:
train_ratio = 0.8
print('* total dataset : ', len(paths))
print('* train ratio : ', train_ratio)

n_train = int(len(paths) * train_ratio)
paths_train = paths[:n_train]
paths_test = paths[n_train:]
print('* n train : ', len(paths_train))
print('* n test : ', len(paths_test))


* total dataset :  39209
* train ratio :  0.8
* n train :  31367
* n test :  7842


In [16]:
paths_train[:3]

['Train/20/00020_00000_00000.png\n',
 'Train/20/00020_00000_00001.png\n',
 'Train/20/00020_00000_00002.png\n']

* `train.txt` 생성

In [17]:
with open('train.txt', 'w') as train_txt:
  for path in paths_train:
    train_txt.write(path)
  print("* create 'train.txt")

* create 'train.txt


* `test.txt` 생성

In [18]:
with open('test.txt', 'w') as test_txt:
  for path in paths_test:
    test_txt.write(path)
  print("* create 'test.txt")

* create 'test.txt


### 3. `.names`
클래스의 이름이 들어있습니다.

In [19]:
with open('classes.names', 'w') as names:
  for class_name in class_names:
    names.write(class_name + '\n')
  print("* create 'class_name'")

* create 'class_name'


### 4. `.data`

학습에 대한 기본적인 정보가 들어있습니다.

* classes : 클래스 개수
* train: 훈련 데이터 (txt 파일)
* valid: 테스트 데이터 (txt 파일)
* names: 클래스 이름 (names 파일)
* backup: 백업 폴더

In [20]:
with open('traffic.data', 'w') as data:
  data.write('classes = 4\n')
  data.write('train = train.txt\n')
  data.write('valid = test.txt\n')
  data.write('names = classes.names\n')
  data.write('backup = backup')

  print("* create 'traffic.data'")

* create 'traffic.data'


* backup폴더 생성

In [21]:
! mkdir backup

### 5. `.cfg`
* .cfg 파일에 max_batches, steps 중요
* train

```
[net]
batch=32
subdivisions=16
width=416
height=416
```

* test

```
[net]
batch=1
subdivisions=1
width=416
height=416
```


## 모델 가져오기
---


https://pjreddie.com/darknet/yolo/

* `darknet` : 모델 구조
* `weight` : pre-train된 가중치 값

### 1. `darknet` 

In [22]:
! git clone https://github.com/pjreddie/darknet

Cloning into 'darknet'...
remote: Enumerating objects: 5943, done.
remote: Total 5943 (delta 0), reused 0 (delta 0), pack-reused 5943
Receiving objects: 100% (5943/5943), 6.37 MiB | 31.34 MiB/s, done.
Resolving deltas: 100% (3927/3927), done.


In [23]:
! rm -rf ./darknet

In [46]:
! ls -la

total 399480
drwxr-xr-x  1 root root      4096 Jan 17 02:55 .
drwxr-xr-x  1 root root      4096 Jan 17 02:53 ..
drwxr-xr-x  2 root root      4096 Jan 17 02:54 backup
-rw-r--r--  1 root root        35 Jan 17 02:54 classes.names
drwxr-xr-x  4 root root      4096 Jan  7 14:33 .config
-rw-r--r--  1 root root   3213624 Jan 17 02:55 darknet
-rw-r--r--  1 root root 162482580 Mar 25  2018 darknet53.conv.74
drwxr-xr-x  1 root root      4096 Jan  7 14:33 sample_data
-rw-r--r--  1 root root    243102 Jan 17 02:54 test.txt
-rw-r--r--  1 root root        84 Jan 17 02:54 traffic.data
-rw-r--r--  1 root root      8328 Jan 17 02:55 traffic-train-yolo.cfg
drwxr-xr-x 45 root root      4096 Jan 17 02:53 Train
-rw-r--r--  1 root root    957707 Jan 17 02:54 train.txt
-rw-------  1 root root 242113694 Jan 17 02:53 Train.zip


In [47]:
! chmod +x ./darknet

In [48]:
! ls -la

total 399480
drwxr-xr-x  1 root root      4096 Jan 17 02:55 .
drwxr-xr-x  1 root root      4096 Jan 17 02:53 ..
drwxr-xr-x  2 root root      4096 Jan 17 02:54 backup
-rw-r--r--  1 root root        35 Jan 17 02:54 classes.names
drwxr-xr-x  4 root root      4096 Jan  7 14:33 .config
-rwxr-xr-x  1 root root   3213624 Jan 17 02:55 darknet
-rw-r--r--  1 root root 162482580 Mar 25  2018 darknet53.conv.74
drwxr-xr-x  1 root root      4096 Jan  7 14:33 sample_data
-rw-r--r--  1 root root    243102 Jan 17 02:54 test.txt
-rw-r--r--  1 root root        84 Jan 17 02:54 traffic.data
-rw-r--r--  1 root root      8328 Jan 17 02:55 traffic-train-yolo.cfg
drwxr-xr-x 45 root root      4096 Jan 17 02:53 Train
-rw-r--r--  1 root root    957707 Jan 17 02:54 train.txt
-rw-------  1 root root 242113694 Jan 17 02:53 Train.zip


### 2. `weight`

```
!wget https://pjreddie.com/media/files/darknet53.conv.74
```

In [25]:
!wget https://pjreddie.com/media/files/darknet53.conv.74

--2022-01-17 02:54:31--  https://pjreddie.com/media/files/darknet53.conv.74
Resolving pjreddie.com (pjreddie.com)... 128.208.4.108
Connecting to pjreddie.com (pjreddie.com)|128.208.4.108|:443... connected.
HTTP request sent, awaiting response... 200 OK
Length: 162482580 (155M) [application/octet-stream]
Saving to: ‘darknet53.conv.74’

darknet53.conv.74   100%[===================>] 154.96M  46.3MB/s    in 3.7s    

2022-01-17 02:54:35 (41.8 MB/s) - ‘darknet53.conv.74’ saved [162482580/162482580]



## 훈련
---


In [ ]:
! ./darknet detector train traffic.data traffic-train-yolo.cfg darknet53.conv.74 -dont_show

traffic-train-yolo
layer     filters    size              input                output
   0 conv     32  3 x 3 / 1   416 x 416 x   3   ->   416 x 416 x  32 0.299 BF
   1 conv     64  3 x 3 / 2   416 x 416 x  32   ->   208 x 208 x  64 1.595 BF
   2 conv     32  1 x 1 / 1   208 x 208 x  64   ->   208 x 208 x  32 0.177 BF
   3 conv     64  3 x 3 / 1   208 x 208 x  32   ->   208 x 208 x  64 1.595 BF
   4 Shortcut Layer: 1
   5 conv    128  3 x 3 / 2   208 x 208 x  64   ->   104 x 104 x 128 1.595 BF
   6 conv     64  1 x 1 / 1   104 x 104 x 128   ->   104 x 104 x  64 0.177 BF
   7 conv    128  3 x 3 / 1   104 x 104 x  64   ->   104 x 104 x 128 1.595 BF
   8 Shortcut Layer: 5
   9 conv     64  1 x 1 / 1   104 x 104 x 128   ->   104 x 104 x  64 0.177 BF
  10 conv    128  3 x 3 / 1   104 x 104 x  64   ->   104 x 104 x 128 1.595 BF
  11 Shortcut Layer: 8
  12 conv    256  3 x 3 / 2   104 x 104 x 128   ->    52 x  52 x 256 1.595 BF
  13 conv    128  1 x 1 / 1    52 x  52 x 256   ->    52 x  52 x 